# Working With Matrix Market Corpora in Gensim

2019-02-21

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import pandas as pd
import pickle
import random

from gensim.corpora import MmCorpus

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


## Corpus Data

For this example, I've acquired the most recent corpus, *arxiv-full-10M-spacy.tar*, which includes five files:

  1. A log file from the corpus creation process
  2. The Matrix Market corpus (`.mm`)
  3. The Matrix Market index file (`.mm.index`)
  4. The token dictionary (`.mm.dictionary.cpickle`)
  5. The document id <--> document name mappings (`.mm.metadata.cpickle`)

In [3]:
!ls -lh ../../corpora/arxiv/arxiv-full-10M-spacy

total 18508472
-rw-r--r--  1 tshafer  staff   102K Feb 19 18:33 2019-02-19-arxiv-full-10M.log
-rw-r--r--  1 tshafer  staff   8.6G Feb 19 18:33 arxiv-full-10M-spacy.mm
-rw-r--r--  1 tshafer  staff   185M Feb 19 18:33 arxiv-full-10M-spacy.mm.dictionary.cpickle
-rw-r--r--  1 tshafer  staff   6.6M Feb 19 18:33 arxiv-full-10M-spacy.mm.index
-rw-r--r--  1 tshafer  staff    32M Feb 19 18:33 arxiv-full-10M-spacy.mm.metadata.cpickle


## Dictionary and Metadata

The dictionary and metadata are available as pickled Python objects.

In [4]:
def unpickle(path):
    with open(path, 'rb') as fil:
        return pickle.load(fil)

In [5]:
meta = unpickle('../../corpora/arxiv/arxiv-full-10M-spacy/arxiv-full-10M-spacy.mm.metadata.cpickle')
[meta[i] for i in range(5)]

[('math9209221',),
 ('math9209214',),
 ('hep-th9207066',),
 ('cond-mat9210006',),
 ('math9209210',)]

In [6]:
dictionary = unpickle('../../corpora/arxiv/arxiv-full-10M-spacy/arxiv-full-10M-spacy.mm.dictionary.cpickle')
[dictionary[i] for i in range(5)]

["'s", 'a.', 'a.m.s.', 'absolutely', 'accord']

Both the metadata and the dictionary are indexed like (or simply are) Python dictionaries.

In [7]:
type(meta)

dict

In [8]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

## Corpus

The *gensim* package is set up to stream the Matrix Market corpus, rather than loading it all into memory. (Of course, the dictionary itself is pretty large and was loaded into memory.)

In [9]:
corpus = MmCorpus('../../corpora/arxiv/arxiv-full-10M-spacy/arxiv-full-10M-spacy.mm')

INFO:gensim.corpora.indexedcorpus:loaded corpus index from ../../corpora/arxiv/arxiv-full-10M-spacy/arxiv-full-10M-spacy.mm.index
INFO:gensim.corpora._mmreader:initializing cython corpus reader from ../../corpora/arxiv/arxiv-full-10M-spacy/arxiv-full-10M-spacy.mm
INFO:gensim.corpora._mmreader:accepted corpus with 1062145 documents, 5793463 features, 662990289 non-zero entries


This corpus format allows us to iterate through the corpus; it also allows for random seeks:

In [10]:
# Iterate through the corpus
for i, doc in enumerate(corpus):
    print(f'Doc no. {i+1:>2d}: {len(doc):>3d} unique tokens and {int(sum(x[1] for x in doc)):>4d} total tokens')
    if i == 9:
        break

Doc no.  1:   0 unique tokens and    0 total tokens
Doc no.  2: 593 unique tokens and 2889 total tokens
Doc no.  3: 330 unique tokens and 1425 total tokens
Doc no.  4: 322 unique tokens and 1360 total tokens
Doc no.  5: 182 unique tokens and  743 total tokens
Doc no.  6: 609 unique tokens and 1927 total tokens
Doc no.  7: 408 unique tokens and  786 total tokens
Doc no.  8: 544 unique tokens and 1523 total tokens
Doc no.  9: 288 unique tokens and  650 total tokens
Doc no. 10: 651 unique tokens and 3052 total tokens


In [11]:
# Access part of one document directly
corpus[10][:10]

[(0, 1.0),
 (1, 2.0),
 (5, 2.0),
 (14, 1.0),
 (16, 1.0),
 (19, 3.0),
 (21, 1.0),
 (24, 3.0),
 (27, 1.0),
 (33, 3.0)]

Each document is in bag-of-words format: The document is represented by a list of tuples:

    [(token_id, count), (token_id, count), ...]

## Corpus Operations

I tried to remove the most common tokens during the corpus creation, but we might still want to remove others. We can map this out by iterating over the entire corpus. (Here, I'm only taking a random 10,000 documents.)

In [12]:
random.seed(2**10)

tokens = []
for depth, i in enumerate(random.sample(range(len(corpus)), 10000)):
    arxiv_id = meta[i][0]
    doc = corpus[i]
    tokens += [
        {'doc': arxiv_id, 'token': dictionary[token], 'count': int(count)}
        for token, count in doc
    ]
    if depth%1000 == 999:
        logging.info('have seen %05d documents', depth+1)

tokens = pd.DataFrame(tokens)

INFO:root:have seen 01000 documents
INFO:root:have seen 02000 documents
INFO:root:have seen 03000 documents
INFO:root:have seen 04000 documents
INFO:root:have seen 05000 documents
INFO:root:have seen 06000 documents
INFO:root:have seen 07000 documents
INFO:root:have seen 08000 documents
INFO:root:have seen 09000 documents
INFO:root:have seen 10000 documents


In [13]:
_counts = tokens.groupby('token')['count'].agg('sum').to_frame().reset_index()
_docs = tokens.groupby('token')['doc'].nunique().to_frame().reset_index()
tokens = _counts.merge(_docs, how='outer', on='token').sort_values('count', ascending=False)

In [14]:
tokens

token   count   doc
238440                use  162583  9502
79910            function  121037  8360
145044              model  119678  7144
191099             result  110005  9397
32089                case  106147  8906
127255                let  104842  5775
76938              follow   99728  8872
215602              state   99609  7202
74259               field   95343  6464
239716              value   91685  8394
228728               time   88967  7439
172436               phys   88800  4794
204969                set   85094  7883
65623              energy   84515  5584
110512                 j.   83956  7120
174945              point   83220  7997
163448              order   82027  8494
51338              define   78350  7993
160750             obtain   74244  8617
226123               term   70656  7513
159602             number   70068  8176
212569              space   69857  6885
670                    a.   67677  6955
124748              large   67577  7970
77368                form   67226  8013
42180            consider   66477  8470
132659                 m.   65959  6857
167445          parameter   58845  7004
227348             theory   57598  6552
68506                  et   55788  4844
...                   ...     ...   ...
158571             nossal       1     1
158589            nostrum       1     1
158587             nostro       1     1
158586            nostril       1     1
158585           nostrify       1     1
158584     nostrification       1     1
59949            dsormais       1     1
158582        nostradamus       1     1
158581             nostos       1     1
59952             dspace/       1     1
158577         nostandard       1     1
59953                dspg       1     1
59955          dsph.found       1     1
59958   dspossibleenergie       1     1
158569         nosratpour       1     1
59964               dsrdn       1     1
59960                dsps       1     1
158567            nosplit       1     1
158566             nospin       1     1
158565              nosov       1     1
158564         nosonovsky       1     1
59961               dsqrt       1     1
158562         nosocomial       1     1
158561          noskowicz       1     1
158559              nosin       1     1
158557         noshowpacs       1     1
158556         noshowkeyw       1     1
158555            nosheen       1     1
158554        noshearedge       1     1
255342               ﬂora       1     1

[255343 rows x 3 columns]

Anything appearing in a single document is no good by definition, unless we really think themes can be generated from only 1 document out of 1.1 million. We should also (probably) remove tokens containing just a single letter or a single number. Depending on how hard we want to make things, we could also remove tokens like "phys", "rev", "lett", and so on because these are common journal names.